In [3]:
!pip install transformers
from transformers import PreTrainedTokenizerFast

U_TKN = '<usr>'
S_TKN = '<sys>'
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 

tok = TOKENIZER


In [4]:
import pandas as pd
import numpy as np

df=pd.read_csv('/content/emotion_pre.csv')

,sentence,emotion
0,죽여버리고 싶다,분노
1,화가 치밀어 올라,분노
2,이런 정말 화나는걸??,분노
3,아 개짜증나,분노
4,아 죽여버리고싶어,분노
...,...,...
106,웃겨,기쁨
107,행복하다,기쁨
108,지금 행복해요,기쁨
109,좋아서 죽겠어요,기쁨


In [5]:
def data_to_x_data(df):
  max_ids=0
  tok_list=[]
  for i,j in df.iterrows():
    sent_tokens = tok.tokenize(j[0])
    token=tok.convert_tokens_to_ids(sent_tokens)
    tok_list.append(token)

    p=max(token)
    if p>max_ids:
      max_ids=p

  def make_data(datas,max_ids):
    zero_list=np.zeros(max_ids+1)
    for i in datas:
      zero_list[i]=1
    return zero_list

  x_data=[]
  for i in tok_list:
    x_data.append(make_data(i,max_ids))
  print(x_data)

  x_data=np.array(x_data)
  return x_data,max_ids
  
x_data,max_ids=data_to_x_data(df)

[array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 

In [6]:
from sklearn import preprocessing

def encode_features(dataDF):
  oh_encoder=preprocessing.OneHotEncoder()
  le = preprocessing.LabelEncoder()
  le=le.fit(dataDF)
  dataDF=le.transform(dataDF)
  labels = dataDF.reshape(-1,1)
  oh_encoder.fit(labels)
  dataDF = oh_encoder.transform(labels)
  return dataDF
  
y_data=encode_features(df['emotion'])
y_data=np.array(y_data.toarray())


In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.2,random_state=11)
print(x_train.shape)

(88, 50433)


In [8]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
import tensorflow as tf

inputs=x_data.shape[1]
outputs=y_data.shape[1]
model = models.Sequential()
model.add(layers.Dense(1024, activation='relu', input_shape=(inputs,)))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(outputs, activation=tf.nn.softmax))

sgd=optimizers.SGD(lr=0.01)
model.compile(optimizer='Adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])


model.fit(x_train, y_train, epochs=6)
model.save("my_model.h5")

Epoch 1/6
3/3 [==============================] - 2s 288ms/step - loss: 1.0979 - accuracy: 0.3519
Epoch 2/6
3/3 [==============================] - 1s 285ms/step - loss: 1.0704 - accuracy: 0.5220
Epoch 3/6
3/3 [==============================] - 1s 278ms/step - loss: 0.9855 - accuracy: 0.7461
Epoch 4/6
3/3 [==============================] - 1s 283ms/step - loss: 0.7997 - accuracy: 0.8079
Epoch 5/6
3/3 [==============================] - 1s 297ms/step - loss: 0.5242 - accuracy: 0.9112
Epoch 6/6
3/3 [==============================] - 1s 291ms/step - loss: 0.2267 - accuracy: 0.9904


In [9]:
pre=model.predict(x_test)
corect=0
sum=0
for i,j in enumerate(pre):
  predict=np.argmax(j)
  if np.argmax(y_test[i])==predict:
    corect+=1
  sum+=1
acc=corect/sum
print("모델 정확도 :",acc)
  

모델 정확도 : 0.6521739130434783


In [10]:
class emotion_analysis():
  def __init__(self,path,max_ids):
    self.model=load_model(path)
    self.max_ids=max_ids
  def pred(self,words):
    data=self.word_to_encoded(words)
    pre=self.model.predict(data)
    return np.argmax(pre)

  def word_to_encoded(self,words):
    toks=tok.tokenize(words)
    tok_list=token=tok.convert_tokens_to_ids(toks)
    zero_list=np.zeros(self.max_ids+1)

    for i in tok_list:
      zero_list[i]=1
      
    x_data=[]
    x_data.append(zero_list)
    x_data=np.array(x_data)
    return x_data
  

In [12]:
from keras.models import load_model

mod=emotion_analysis('/content/my_model.h5',max_ids)

In [13]:
print(mod.pred('안녕'))

0


In [15]:
!cp -r /content/my_model.h5 /content/drive/MyDrive